# Pre-processing fighter statistics

In [81]:
# import

import datetime as DT
import io
import numpy as np
import pandas as pd
import re
from distutils.log import error


In [82]:
# load csv file

df = pd.read_csv('fighter_scraper/fighterstemp.csv')

In [83]:
df=df.drop_duplicates()

In [84]:
df.dtypes

name           object
win             int64
loss            int64
tie            object
height_ft      object
weight_lbs     object
reach_inch     object
stance         object
dob            object
slpm          float64
str_acc        object
sapm          float64
str_def        object
tdavg         float64
tdacc          object
tddef          object
subavg        float64
dtype: object

In [85]:
df.isna().sum()

name            0
win             0
loss            0
tie             0
height_ft       0
weight_lbs      0
reach_inch      0
stance        306
dob             0
slpm            0
str_acc         0
sapm            0
str_def         0
tdavg           0
tdacc           0
tddef           0
subavg          0
dtype: int64

In [91]:
# feature engineering

df['w/l_ratio'] = (df['win'] / df['loss']).round(2)

In [43]:
# turn into string

#df['dob'] = df['dob'].astype(str)

In [87]:
#get age

now = pd.Timestamp('now')

df['dob'] = [x.replace(",", "") for x in df['dob']]

df['dob'] = pd.to_datetime(df['dob'], errors='coerce')

df['dob'] = df['dob'].where(df['dob'] < now, df['dob'] -  np.timedelta64(100, 'Y'))
df['age'] = (now - df['dob']).astype('<m8[Y]')
df=df.drop(['dob'], axis=1)


In [88]:
#feet to inches
r = re.compile(r"([0-9]+)' ([0-9]*\.?[0-9]+)\"")
def get_inches(el):
    m = r.match(el)
    if m == None:
        return float('NaN')
    else:
        return int(m.group(1))*12 + float(m.group(2))
    

#run function to create new column
df['inches'] = df['height_ft'].apply(get_inches)

#drop old column

df = df.drop(['height_ft'], axis=1)

In [89]:
# get rid of % 
# do this is scraping if can 

df['str_acc'] = [x.replace("%", "") for x in df['str_acc']]
df['str_def'] = [x.replace("%", "") for x in df['str_def']]
df['tdacc'] = [x.replace("%", "") for x in df['tdacc']]
df['tddef'] = [x.replace("%", "") for x in df['tddef']]

In [101]:
# converting to proper data types

df = df.convert_dtypes()

df.dtypes # the way the data was scraped some nulls were scraped as strings

name           string
win             Int64
loss            Int64
tie            string
weight_lbs     string
reach_inch     string
stance         string
slpm          Float64
str_acc        string
sapm          Float64
str_def        string
tdavg         Float64
tdacc          string
tddef          string
subavg        Float64
w/l_ratio     Float64
age             Int64
inches          Int64
dtype: object

In [103]:
df.columns

Index(['name', 'win', 'loss', 'tie', 'weight_lbs', 'reach_inch', 'stance',
       'slpm', 'str_acc', 'sapm', 'str_def', 'tdavg', 'tdacc', 'tddef',
       'subavg', 'w/l_ratio', 'age', 'inches'],
      dtype='object')

In [105]:
df.isnull().sum()

name            0
win             0
loss            0
tie             0
weight_lbs      0
reach_inch      0
stance        306
slpm            0
str_acc         0
sapm            0
str_def         0
tdavg           0
tdacc           0
tddef           0
subavg          0
w/l_ratio       1
age           427
inches        127
dtype: int64

In [118]:
temp

tdacc        NaN
tddef        NaN
str_def      NaN
str_acc      NaN
reach_inch   NaN
dtype: float64